In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.python.ops import rnn_cell
import math

In [8]:
#weight 매트릭스 초기화 함수
#https://github.com/therne/dmn-tensorflow 참고
def weight(name, shape, init='he', range=None):
    initializer = tf.constant_initializer()
    if init == 'xavier':
        fan_in, fan_out = _get_dims(shape)
        range = math.sqrt(6.0 / (fan_in + fan_out))
        initializer = tf.random_uniform_initializer(-range, range)

    elif init == 'he':
        fan_in, _ = shape
        std = math.sqrt(2.0 / fan_in)
        initializer = tf.random_normal_initializer(stddev=std)

    elif init == 'normal':
        initializer = tf.random_normal_initializer(stddev=0.1)

    elif init == 'uniform':
        if range is None:
            raise ValueError("range must not be None if uniform init is used.")
        initializer = tf.random_uniform_initializer(-range, range)

    var = tf.get_variable(name, shape, initializer=initializer)
    tf.add_to_collection('l2', tf.nn.l2_loss(var))  # Add L2 Loss
    return var

##문자열 처리 함수

In [9]:
#문장 부호 제거
def remove_marks(strIn):
    marks = ['.', ',', '-', '!', '?', '\"']
    
    result = strIn
    for mark in marks:
        result = result.replace(mark, '')
    
    result = result.strip()
    return result


#데이터 읽어오기
def read_story():
    sentences = []
    labels = []
    with open('stories.txt', 'r') as f:
        for line in f: #문장을 한 줄씩 읽어서
            chunks = line.split('\t') #탭을 기준으로 분리. 앞쪽이 문장, 뒤쪽은 클래스 레이블
            sentences.append( remove_marks(chunks[0]) ) #문장에서 문장부호 제거
            labels.append(int(chunks[1]))
            
    return sentences, labels

#본 예제에서는 GloVe 벡터를 단어 representation로 사용
#http://nlp.stanford.edu/projects/glove/ 에서 다운
def load_glove(dim):
    word2vec = {} 
    with open( "../glove.6B." + str(dim) + "d.txt") as f: 
        for line in f:
            l = line.split()
            word2vec [l[0]] = map( float, l[1:])
    return word2vec

#GloVe에 미리 정의되지 않은 단어에 대해서는, 랜덤하게 representation 생성
def create_vector(word, word2vec, word_vector_size):
    vector = np.random.uniform( 0.0, 1.0, (word_vector_size,) )
    word2vec[word] = vector
    return vector

#문장에서 하나의 단어를 입력으로 받아 GloVe vector로 바꾸는 함수
def process_word(word, word2vec, vocab, ivocab, word_vector_size):
    if not word in word2vec:
        create_vector(word, word2vec, word_vector_size)
    if not word in vocab:
        next_index = len(vocab)
        vocab[word] = next_index
        ivocab[next_index] = word
        
    return word2vec[word]

#문장을 받아서, 띄어쓰기 단위로 분리하고 GloVe representation으로 바꾸어 전체 입력 데이터를 만드는 함수
#각 문장의 단어수도 저장
def make_input_vector(sentences, word2vec, vocab, ivocab, word_vector_size):
    inputs = []
    num_words = []
    for line in sentences:
        inp = line.lower().split(' ') #띄어쓰기 기준으로 문장 나누기
        inp = [ w for w in inp if len(w) > 0] 
        
        inp_vector = [process_word( word = w,
                                  word2vec = word2vec,
                                  vocab = vocab,
                                  ivocab = ivocab,
                                  word_vector_size = word_vector_size) for w in inp]
        inp_vector = np.vstack((inp_vector))
        #print inp_vector.shape
        inputs.append( inp_vector )
        num_words.append(len(inp))
        #print len(vocab), len(ivocab)
        
    return inputs, num_words

In [10]:
#GloVe representation 의 벡터 사이즈 지정
word_vector_size = 100
assert word_vector_size in [50, 100, 200, 300]

#GloVe 로딩
print 'load_glove'
word2vec = load_glove(word_vector_size)


load_glove


In [11]:
#Data Padding, 가장 긴 문장을 기준으로 zero padding
def process_batch():
        start_index = 0
        end_index = 200
        
        inp = inputs[start_index:end_index]
        num_word = num_words[start_index:end_index]
        label = labels[start_index:end_index]
        
        B, V = batch_size, word_vector_size

        L = 0
        for n in range(B):
            sent_len = len(inp[n])
            L = max(L, sent_len)


        new_input = np.zeros([B, L, V])  # zero padding
        new_labels = []
        new_num_words = []
        for n in range(B):
            new_input[n, :num_word[n]] = inp[n]
            new_labels.append(label[n])
            new_num_words.append(num_word[n])


        return new_input, new_labels, new_num_words

##모델 parameter define

In [12]:
vocab = {}
ivocab = {}

batch_size = 200 
hidden_size = 50 #GRU hidden layer의 노드 수
num_classes = 4
learning_rate = 0.001
sentences, labels = read_story()
inputs, num_words = make_input_vector(sentences, word2vec, vocab, ivocab, word_vector_size)
vocab_size = len(vocab)



IndexError: list index out of range

##RNN 모델 정의

In [13]:
#B: batch size, L: 가장 긴 문장의 길이(=가장 긴 문장에 있는 단어의 수)
#V: GloVe word vector size, h: GRU Hidden Node의 수, A: 데이터에 포함된 단어의 총 갯수
B, L = batch_size, max(num_words)
V, h, A = word_vector_size, hidden_size, vocab_size

inp, len_sentences, label = process_batch()
inp = np.float32(inp)
#feed_dict={inp:new_input, len_sentences:new_num_words, label:new_labels}
"""
inp = tf.placeholder('float32', shape = [B, L, V], name = 'x')
len_sentences = tf.placeholder('int32', shape = [B], name = 'x_len')
label = tf.placeholder('int32', shape=[B], name = 'y')
"""

#GRU Cell 정의
gru = rnn_cell.GRUCell(h)
outputs, final_states = tf.nn.dynamic_rnn(cell = gru, 
                                          inputs = inp, 
                                          sequence_length = len_sentences, 
                                          dtype = tf.float32)    

#GRU hidden 으로부터 클래스를 예측하기 위한 추가 layer
with tf.name_scope('answer'):
    w_a = weight('answer_weight', [h, num_classes] )
    logits = tf.matmul(final_states, w_a)

#Cross entropy로 loss function 정의
with tf.name_scope('loss'):
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits, label)
    loss = tf.reduce_mean(cross_entropy)

#정답과 모델이 예측한 답을 비교하여 정확도 측정
with tf.name_scope('acc'):
    predicts = tf.cast(tf.argmax(logits, 1), 'int32')
    corrects = tf.equal(predicts, labels[:batch_size])
    num_corrects = tf.reduce_sum(tf.cast(corrects, tf.float32))
    accuracy = tf.reduce_mean(tf.cast(corrects, tf.float32))

optimizer = tf.train.AdamOptimizer(learning_rate)
opt_op = optimizer.minimize(loss) #, global_step = self.global_step)

NameError: name 'num_words' is not defined

##Computational Graph Launching, 실행

In [11]:

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    

    for i in range(10):
        sess.run(opt_op)
        a = sess.run(final_states)
        acc_print = sess.run(accuracy)
        loss_print = sess.run(loss)
        